In [2]:
from setup_general import *

# Finish unit translation/ unification &  values to float

In [184]:
data = prep.copy()
data['value'] = data['value'].apply(lambda x: float(x.replace(',', '.')) if type(x) == str else x)
data.to_csv('data/prep.csv')

In [210]:
data = prep.copy()

# e.g wrong translation
data['unit'] = data['unit'].replace('e.g','mm')
data['unit'] = data['unit'].replace('pc','pieces')
data['unit'] = data['unit'].replace('page','pages')
# unify units
data['unit'] = data['unit'].replace('10 x 15 cm','100 x 150 mm')

data.to_csv('data/prep.csv')

# mm to cm

In [296]:
data = prep.copy()


data['value'] = data.apply(lambda item: item['value'] / 10 if item['unit'] == 'mm' else item['value'], axis=1)
data['unit'] = data['unit'].replace('mm','cm')
data['value'] = pd.to_numeric(data['value'])

data.to_csv('data/prep.csv')

# Combine parameter, unit & w/h values to value

In [304]:
def get_squared(item):
    if ' x ' in item:
        return item + '²'
    else:
        return item

def extract_width_height_from_unit_to_value(item):
    unit = item[0]
    value = item[1]
    if ' x ' in unit:
        split = unit.split(' ')
        x = split[0]
        y = split[2]
        real_unit = split[3]
        real_value = [x,y]        
        return [real_unit, real_value]

    else:
        return [unit, value]

In [306]:
data = prep.copy()

data['unit'] = data['unit'].replace(np.nan,'*')
data['parameter'] = data['parameter'].replace(np.nan,'*')
data['unit'] = data['unit'].apply(lambda x: get_squared(x))
# execution order is important
data['value'] = data.apply(lambda item: extract_width_height_from_unit_to_value(item[['unit','value']])[1], axis=1)
data['unit'] = data.apply(lambda item: extract_width_height_from_unit_to_value(item[['unit','value']])[0], axis=1)
data['parameter_and_unit'] = data['parameter'] + ' IN ' + data['unit']

data.to_csv('data/prep.csv')

# parameter_and_units as single features with respective values

In [316]:
data = prep.copy()

# parameter_and_unit turned into one hot encoded features
data= pd.get_dummies(data, columns=['parameter_and_unit'], prefix='', prefix_sep='')
data.to_csv('data/prep.csv')

In [319]:
# now put values into one hot encoded features
data = prep.copy()

# for all new "parameter with unit" columns put the value in the column where a 1 is - others are 0 and remain 0
for column in data.columns:
    if ' IN ' in column:
        data[column] = data.apply(lambda item: item['value'] * item[column], axis=1)

data.to_csv('data/prep.csv')

# make all size features numeric

In [ ]:
def extract_height_width(item):    
    if type(item) == str:
        # Converting string to list
        res = item.strip('][').split(', ')
        h = float(res[0].strip('\''))
        w = float(res[1].strip('\''))
        return [h,w]

    else:
        return [float('nan'), float('nan')]

In [4]:
data = prep.copy()

for column in data.columns:
    # all the parameter with unit columns that contain arrays that are represeted as strings
    if (' IN ' in column) and (data[column].dtype == object):
        data[column + '_height'] = data.apply(lambda item: extract_height_width(item[column])[0], axis=1)
        data[column + '_width'] = data.apply(lambda item: extract_height_width(item[column])[1], axis=1)

for column in data.columns:
    if (' IN ' in column):
        data[column] = data[column].replace(np.nan,0)

data.to_csv('data/prep.csv')